In [1]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=4)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 653 (delta 47), reused 15 (delta 8), pack-reused 569
Receiving objects: 100% (653/653), 62.41 MiB | 13.14 MiB/s, done.
Resolving deltas: 100% (335/335), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.41.2
Using datasets v2.20.0
Using accelerate v0.32.1
Using sentencepiece v0.1.99
Using seqeval


In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
#!pip install transformers
#!pip install datasets
import pandas as pd
import numpy as np
import os
import torch
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import AutoModel

In [4]:
## util.py - has hard_coded_dictionary
## has preprocess text ... [USR], [BOT] tokens!
import torch
import sys

int2emo = {
    0: "기쁨", 1:"당황", 2:"분노", 3:"불안", 4:"상처", 5:"슬픔", 6:"중립"
}

def preprocess_text(text):
    text = text.split(" [SEP] ")
    result = ""
    for i in range(len(text)):
        if i % 2 == 0:
            if len(result) == 0:
                result += "[USR] " + text[i]
            else:
                result += " [USR] " + text[i]
        else:
            result += " [BOT] " + text[i]
    return result

"""
if __name__ == "__main__":
    text = sys.argv[1]
    print(preprocess_text(text))
    print(int2emo[0])
"""

'\nif __name__ == "__main__":\n    text = sys.argv[1]\n    print(preprocess_text(text))\n    print(int2emo[0])\n'

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


int2emo = {0: "기쁨", 1: "당황",  2: "분노", 3:"불안", 4:"상처", 5: "슬픔", 6: "중립"}
class ElectraSentimentClassifier:
    def __init__(self):
        self.model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name).to(self.device)

    def predict_sentiment(self, conversation):
        inputs = self.tokenizer(conversation, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).item()
        probabilities = torch.nn.functional.softmax(logits, dim=-1)[0].tolist()
        self.print_result(conversation, probabilities, predictions)

        return predictions, probabilities  # Ensure int2emo[predictions] is returned correctly

    def ready_model(self):
        self.model.to(self.device)

    def unload_model(self):
        self.model.cpu()
        torch.cuda.empty_cache()
        del self.model
        self.model = None

    def print_result(self, sentence, probabilities, predictions):
        # 터미널에 출력
        print("==== Model Prediction ====")
        print(f"Input Sentence: {sentence}")
        print("Probabilities:")
        for i, prob in enumerate(probabilities):
            print(f"  {int2emo[i]}: {prob * 100:.1f}%")
        predicted_label = int2emo[predictions]  # Add this line to define predicted_label
        print(f"Predicted Label: {predicted_label} ({predictions})")
        print("==========================")

In [6]:
model = ElectraSentimentClassifier()
model.ready_model()
model.predict_sentiment("[USR] 안녕하세요!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/815k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

==== Model Prediction ====
Input Sentence: [USR] 안녕하세요!
Probabilities:
  기쁨: 57.0%
  당황: 3.0%
  분노: 0.6%
  불안: 0.1%
  상처: 0.0%
  슬픔: 0.6%
  중립: 38.7%
Predicted Label: 기쁨 (0)


(0,
 [0.5696581602096558,
  0.030423378571867943,
  0.005696214735507965,
  0.000829820055514574,
  0.0003184544912073761,
  0.006047842558473349,
  0.38702622056007385])

In [7]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import numpy as np

import os

import logging

# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter("%(message)s")

# Start defining and assigning your handlers here
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logger.addHandler(handler)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 도서 목록 로드
file_path = '/content/도서data.csv'  # 파일 업로드 후 경로 변경 필요
book_list = pd.read_csv(file_path)

# 도서 목록 전처리
book_list['introduction2'] = book_list['introduction2'].apply(lambda x: re.sub('[^ㄱ-ㅣ가-힣 ]',"",x) if isinstance(x, str) else x)

from nltk.corpus import stopwords

def remove_stop_words(word_tokens: list) -> list:

    if not isinstance(word_tokens, list):
        return word_tokens

    k_stopword=pd.read_csv("/content/korean_stopword.csv")
    k_stopword = list(k_stopword['불용어'])

    result = []

    for token in word_tokens:
        if token not in stop_words:
            result.append(token)

    return result

book_list['introduction2'] = book_list.apply(lambda x: remove_stop_words(x['introduction2']), axis=1)
# 감정 분석 클래스 인스턴스 생성
classifier = ElectraSentimentClassifier()

# 도서 소개 텍스트 감정 분석
def analyze_book_emotions(book_list):
    book_emotions = {}
    emotions = []
    for index, row in book_list.iterrows():
        introduction = row['introduction2']
        prediction, _ = classifier.predict_sentiment(introduction)
        book_emotions[row['title']] = int2emo[prediction]
        emotions.append(int2emo[prediction])
    book_list['emotion'] = emotions
    return book_emotions

# 도서 소개 텍스트 감정 분석 결과
book_emotions = analyze_book_emotions(book_list)

# 감정에 따른 도서 추천 함수
def recommend_books_by_emotion(emotion, book_emotions):
    recommended_books = [book for book, book_emotion in book_emotions.items() if book_emotion == emotion]
    return recommended_books

# 사용 예제
diary_entry = "오늘은 너무 행복한 하루였다."
prediction, _ = classifier.predict_sentiment(diary_entry)
emotion = int2emo[prediction]
recommended_books = recommend_books_by_emotion(emotion, book_emotions)

print(f"감정: {emotion}")
print("추천 도서:")
for book in recommended_books:
    print(book)
# 결과를 엑셀 파일로 저장
output_file_path = '/content/book_list_with_emotions.xlsx'
book_list.to_excel(output_file_path, index=False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Probabilities:
  기쁨: 6.5%
  당황: 0.4%
  분노: 0.3%
  불안: 0.0%
  상처: 0.0%
  슬픔: 0.7%
  중립: 92.0%
Predicted Label: 중립 (6)
==== Model Prediction ====
Input Sentence: 에드거 앨런 포의 단편소설은 광기와 일탈을 주제로 비정상적으로 예민하고 파괴 욕구에 시달리는 주인공들이 통제력을
상실하며 벌어지는 사건들을 다룬다 대표작 검은 고양이에서는 주인공이 술에 중독되어 점차 난폭해지며 자신의 고양이를 학대하고 살해하는 이야기를
어셔가의 몰락에서는 정신적 고통을 겪는 친구를 방문한 주인공이 기묘한 저택과 그곳에 사는 사람들의 연관성을 깨닫고 결국 저택이 붕괴하는 이야기를
그린다 포의 작품들은 이성과 비이성의 경계를 넘나드는 인간 내면의 복합성을 드러내며 환상적이고 그로테스크한 요소들로 독자에게 강렬한 인상을
남긴다
Probabilities:
  기쁨: 0.6%
  당황: 2.1%
  분노: 22.2%
  불안: 1.4%
  상처: 1.2%
  슬픔: 72.3%
  중립: 0.2%
Predicted Label: 슬픔 (5)
==== Model Prediction ====
Input Sentence: 양성애자인 주인공이 등장하는 퀴어 로맨스 소설이 출간되자마자 뉴욕타임스 베스트셀러에 올랐다 이 소설은 다양한
미디어에서 호평을 받으며 큰 인기를 끌었고 결국 아마존 스튜디오에서 영화화가 확정되었다 주인공은 독자들에게 용기와 희망을 주는 이야기를 통해
많은 사람들에게 힘을 실어주고 있다
Probabilities:
  기쁨: 85.9%
  당황: 0.6%
  분노: 0.5%
  불안: 0.2%
  상처: 0.1%
  슬픔: 5.5%
  중립: 7.3%
Predicted Label: 기쁨 (0)
==== Model Prediction ====
Input Sentence: 릴케는 독일의 대표적인 시인으로 